In [1]:
import pandas as pd

# import src.sk_ml; reload(src.sk_ml)
# from src.sk_ml import *

import wKit.ML.sk_ml; reload(wKit.ML.sk_ml)
from wKit.ML.sk_ml import *
from wKit.utility.ipynb_helper import multi_column_df_display
from src.constants import fn_target_lts_dc, dir_data
from src.pred_lts import prepare_lts_dataset

%matplotlib inline

In [2]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(True)

In [3]:
fna = 0.0
years = (2014, 2015, 2016, 2017)
scaler_type = 'MinMaxScaler'
total_or_not = 'TOTAL'

In [4]:
datasets = {}
for total_or_not in ['NO_TOTAL', 'TOTAL']:
    print '=' * 20, 'loading data for', total_or_not
    scaler = scaler_by_name(scaler_type)
    ds = prepare_lts_dataset(scaler, fna, years, total_or_not, 'dict')
    datasets[total_or_not] = ds


==================== loading data for NO_TOTAL
loading feature and fill NAN
loading LTS and remove 10
creating train and test set
normalizing X
==================== loading data for TOTAL
loading feature and fill NAN
loading LTS and remove 10
creating train and test set
normalizing X


#  load models

In [5]:
from datetime import datetime as dtm

In [6]:
loaded_models = {}
for total_or_not in ['TOTAL', 'NO_TOTAL']:
    cv_dir = 'data/cross_validation/na_{}-year_{}-total_{}-norm_{}'.format(fna, years, total_or_not, scaler_type)
    print '=' * 20, 'loading cv result from', cv_dir
    
    ds = datasets[total_or_not]
    train_x, train_y = ds['train_x'], ds['train_y']
    models = sk_models(stoplist=())
    params = grid_cv_default_params()

    kind = 'cls'
    
    name_rf = 'RFcls'
    print dtm.now(), 'model =', name_rf
    model_rf, param_rf = models[kind][name_rf], params[kind][name_rf]
    grid_cv_a_model(train_x, train_y, model_rf, param_rf, kind, name_rf, path=cv_dir, verbose=True)
    
    name_gbst = 'GDBcls'
    print dtm.now(), 'model =', name_gbst
    model_gbst, param_gbst = models[kind][name_gbst], params[kind][name_gbst]
    grid_cv_a_model(train_x, train_y, model_gbst, param_gbst, kind, name_gbst, path=cv_dir, verbose=True)

    loaded_models[total_or_not] = {
        'gbst': model_gbst,
        'rf': model_rf,
    }
    print dtm.now(), 'Done'


==================== loading cv result from data/cross_validation/na_0.0-year_(2014, 2015, 2016, 2017)-total_TOTAL-norm_MinMaxScaler
2017-06-14 16:17:47.937000 model = RFcls
loading existing model cls RFcls
fitting model cls RFcls
loaded existing result for model: RFcls
2017-06-14 16:17:52.708000 model = GDBcls
loading existing model cls GDBcls
fitting model cls GDBcls
loaded existing result for model: GDBcls
2017-06-14 16:19:36.777000 Done
==================== loading cv result from data/cross_validation/na_0.0-year_(2014, 2015, 2016, 2017)-total_NO_TOTAL-norm_MinMaxScaler
2017-06-14 16:19:36.778000 model = RFcls
loading existing model cls RFcls
fitting model cls RFcls
loaded existing result for model: RFcls
2017-06-14 16:20:22.317000 model = GDBcls
loading existing model cls GDBcls
fitting model cls GDBcls
loaded existing result for model: GDBcls
2017-06-14 16:27:00.063000 Done


# get confusion matrix and importance

In [7]:
def ana_a_model(ds, model, top=30):
    test_x, test_y, ftr_name = ds['test_x'], ds['test_y'], ds['ftr_name']
    cfsn = confusion_matrix_as_df(model, test_x, test_y, labels=[1,2,3,4])
    imp = show_important_features(model, labels=ftr_name, top=top, set_std=True, show_plt=False)
    return cfsn, imp

In [8]:
ana_res = {}
for total_or_not in ['TOTAL', 'NO_TOTAL']:
    
    ds = datasets[total_or_not]
    ms = loaded_models[total_or_not]
    res = {}
    for key, m in ms.items():
        cfsn, imp = ana_a_model(ds, m, top=20)
        res['cfsn_'+key] = cfsn
        res['imp_'+key] = imp
    ana_res[total_or_not] = res

## display confusion matrix

In [9]:
list_dfs = [
    ana_res['TOTAL']['cfsn_rf'],
    ana_res['NO_TOTAL']['cfsn_rf'],
    ana_res['TOTAL']['cfsn_gbst'],
    ana_res['NO_TOTAL']['cfsn_gbst'],
]
multi_column_df_display(list_dfs)

## analysis based on confusion matrix

In [10]:
datasets['TOTAL'].keys()

['ftr_name', 'train_x', 'train_y', 'col2code', 'test_y', 'test_x']

In [11]:
t = 'NO_TOTAL'
fitted_model = loaded_models[t]['rf']
test_x = datasets[t]['test_x']
test_x = pd.DataFrame(test_x, columns=datasets[t]['ftr_name'])
test_y = datasets[t]['test_y']
imp_rf = ana_res[t]['imp_rf'].label.tolist()

In [12]:
import wKit.stat.cmp; reload(wKit.stat.cmp)
import wKit.ML.sk_ml; reload(wKit.ML.sk_ml)
from wKit.ML.sk_ml import *
from wKit.stat.cmp import cmp_rank_list

In [13]:
target_tp = [(2, 1), (2, 2)]
df = ftr_diff_stat_test(fitted_model, test_x, test_y, target_tp)
mask = (df.annova1way_siglv!='') | (df.kruskal_signlv!='')
true2_pred12 = df[mask].mean_dff.reset_index().values
print df[mask].shape
# df[mask]

d:\my_toolkit\wKit\ML\sk_ml.py:472: UserWarning:

test y is intable, but type of elements are not all int

D:\Program Files\Anaconda2\lib\site-packages\scipy\stats\stats.py:2968: RuntimeWarning:

invalid value encountered in double_scalars



(166, 7)


In [14]:
target_tp = [(2, 1), (1, 1)]
df = ftr_diff_stat_test(fitted_model, test_x, test_y, target_tp)
mask = (df.annova1way_siglv!='') | (df.kruskal_signlv!='')
true12_pred1 = df[mask].mean_dff.reset_index().values
print df[mask].shape
# df[mask]

(95, 7)


t2p1 is more similar to t1p1 than to t2p2:

1. Find features that can differentiate (t2p1, t1p1) and (t2p1, t2p2), i.e. features with significant difference(anova or kruskal) between (t2p1, t1p1) and (t2p1, t2p2)   --> (t2p1, t2p2) has more such features than (t2p1, t1p1) (33 vs 15, or 162 vs 100)

2. mean_dff of (t2p1, t2p2) is much larger than of (t2p1, t1p1)


In [23]:
cmp_rank_list(true12_pred1, true2_pred12, imp_rf)\
# .sort_values('score_change%')
# .sort_values('gold_rank')


item gold_rank gold_change  \
0                          DIRECTIONALITY_Bi-direction                         
1                               DIRECTIONALITY_one-way                         
2                                        STREETTYPE_ST                         
3                                   ud_clo_cntr_SgAsNd        11          +8   
4                                    d_clo_cntr_SgAsNd        14         +10   
5                                   ud_far_cntr_SgAsNd        13          +8   
6                                    d_node_ecc_SgAsNd         6           0   
7                                   ud_node_ecc_SgAsNd         9          +2   
8                                   cycling facilities                         
9                                        STREETTYPE_PL                         
10                                      transportation        18          +8   
11                                OFFENSE_THEFT F/AUTO                         
12                     FAILURE TO DISPLAY CURRENT TAGS                         
13    RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT                         
14                           FAILURE TO SECURE DC TAGS                         
15                      FAIL TO REPORT FOR INSPECTION.                         
16       RESIDENTIAL PMT PKG 2ND OFFENSE CALENDAR YEAR                         
17                                        SIGN REPLACE                         
18                                      NO FRONT TAGS.                         
19                                 USERTYPE_Pedestrian                         
20               FIRSTHARMFULEVENTSPECIFICS_Pedestrian                         
21                                         retail shop         5         -16   
22     FIRSTHARMFULEVENTSPECIFICS_Parked Motor Vehicle                         
23                                    ud_bridge_SgAsNd                         
24                                  ud_deg_cntr_SgAsNd                         
25       RESIDENTIAL PMT PKG 3RD OFFENSE CALENDAR YEAR                         
26        FAIL TO DISPLAY CURRENT INSPECTION STICKER_y                         
27          FAIL TO PROPERLY AFFIX VALIDATION STICKERS                         
28                                      cycle_lane_one                         
29                                           is_shared                         
..                                                 ...       ...         ...   
186                               RESIDENT PARKING PER                         
187                                 ud_btw_cntr_SgAsEg        19               
188               NOT PARKED IN PARKING METER SPACE BE                         
189                       FAIL TO DISPLAY CURRENT TAGS                         
190        OPERATING VEHICLE LEARNER PERMIT NO LICENSE                         
191                                     ANC RESOLUTION                         
192                                     UNSAFE TIRE(S)                         
193             FAIL TO PROVIDE A VALID DRIVER LICENSE                         
194                                      Climbing Lane                         
195                    FAIL TO SLOW AT AN INTERSECTION                         
196                                 ud_btw_cntr_SgAsNd                         
197                            BIKE IMPROPER EQUIPMENT                         
198                           PASSING VEHICLE ON RIGHT                         
199   VEH >15 PASS, PUBLIC ST FRT DWELLING, SCH, HOSP,                         
200            UNREGISTERED VEHICLE, MORE THAN 30 DAYS                         
201        DRIVE TOO SLOWLY TO IMPEDE OR BLOCK TRAFFIC                         
202     RIDE BIKE/PMD SO AS TO CREATE A DRIVING HAZARD                         
203                         UNLICENSED NON-DC OPERATOR                         
204                              COVERED LICENSE P

In [16]:
cmp_rank_list(true2_pred12, true12_pred1, imp_rf)\
.sort_values('gold_rank')

item gold_rank gold_change  \
70   FIRSTHARMFULEVENTSPECIFICS_Motor Vehicle In Tr...         0         -70   
73            CRASHEVENTTYPES_COLLISION_WITH_NON_FIXED         1         -72   
126                                 Existing Bike Lane         3               
71              CRASHEVENTTYPES_ PENDING_INVESTIGATION         4         -67   
108                                        retail shop         5               
0                                    d_node_ecc_SgAsNd         6          +6   
178                                       SHAPE_Length         7               
119                                TRAFFIC SIGNAL MAIN         8               
101                                 ud_node_ecc_SgAsNd         9               
98                                  ud_clo_cntr_SgAsNd        11               
100                                 ud_far_cntr_SgAsNd        13               
99                                   d_clo_cntr_SgAsNd        14               
78                                   d_btw_cntr_SgAsNd        15         -63   
168                                  d_far_cntr_SgAsNd        16               
80                                   d_btw_cntr_SgAsEg        17         -63   
26                                      transportation        18          -8   
38                                  ud_btw_cntr_SgAsEg        19         -19   
142  PARK ON SIDEWALK OR AREA RESERVED FOR PEDESTRIANS                         
143                   FIRSTHARMFULEVENTSPECIFICS_Fence                         
144    FAIL TO PAY ATTENTION WHILE OPERATING A VEHICLE                         
145           FAIL TO YIELD RIGHT OF WAY TO PEDESTRIAN                         
156                        OFFENSE_MOTOR VEHICLE THEFT                         
153                          COLLIDING WITH PEDESTRIAN                         
155                        PARK OVERTIME IN TIMED ZONE                         
147  DISTRACTED DRIVING USING CELL PHONE, OTHER DEVICE                         
148  FIRSTHARMFULEVENTSPECIFICS_Other Post, Pole Or...                         
141                                                art                         
149                                          residence                         
150                        NOT PARKED IN A METER SPACE                         
151                              PARK AT EXPIRED METER                         
..                                                 ...       ...         ...   
110                                   ud_bridge_SgAsNd                         
111                                 ud_deg_cntr_SgAsNd                         
96                              DIRECTIONALITY_one-way                         
113         FAIL TO PROPERLY AFFIX VALIDATION STICKERS                         
83                                                food                         
81                VIOLATION OF THE CHILD RESTRAINT ACT                         
54                EXPIRED INSPECTION REJECTION STICKER                         
55                                  ud_btw_cntr_SgAsNd                         
56             PARK WITHIN 5 FEET OF DRIVEWAY OR ALLEY                         
57                             BIKE IMPROPER EQUIPMENT                         
58                            PASSING VEHICLE ON RIGHT                         
59                                       NO REAR TAGS.                         
60    VEH >15 PASS, PUBLIC ST FRT DWELLING, SCH, HOSP,                         
61             UNREGISTERED VEHICLE, MORE THAN 30 DAYS                         
62         DRIVE TOO SLOWLY TO IMPEDE OR BLOCK TRAFFIC                         
63      RIDE BIKE/PMD SO AS TO CREATE A DRIVING HAZARD                         
82   ABANDON VEHICLE ON PUBLIC/PRIVATE PROPERTY VIOL 3                         
64    PARKED IN DRIVEWAY OR ALLEY TO OBSTRUCT SIDEWALK                         
66          FAIL TO DISPLAY PROOF OF VEHICLE INSUR

## plot diff feature

In [17]:
import wKit.vis.plotly_fig; reload(wKit.vis.plotly_fig)
from wKit.vis.plotly_fig import *

In [18]:
target_tp = [(2,2), (2, 1), (1, 1)]
indices = get_idx_by_true_pred(fitted_model, test_x, test_y, target_tp)

In [24]:
topN = 5
arrs = []
names = []
for col in true2_pred12[:topN, 0]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = box(arrs, names)
iplot(fig)

In [25]:
topN = 5
arrs = []
names = []
for col in true2_pred12[:topN, 0]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = hist(arrs, names)
iplot(fig)

In [26]:
arrs = []
names = []
for col in imp_rf[:topN]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = box(arrs, names)
iplot(fig)

In [27]:
arrs = []
names = []
for col in imp_rf[:topN]:
    for tp in target_tp:
        arrs.append(test_x.iloc[indices[tp]][col].values)
        names.append(col[:30]+'_'+str(tp))
fig = hist(arrs, names)
iplot(fig)

# display feature importances

In [32]:
list_dfs = [
    res['TOTAL']['imp_rf'],
    res['TOTAL']['imp_gbst'],
]
multi_column_df_display(list_dfs, cols=2)

In [33]:
list_dfs = [
    res['NO_TOTAL']['imp_rf'],
    res['NO_TOTAL']['imp_gbst'],
]
multi_column_df_display(list_dfs, cols=2)

# bar chart feature importances

In [82]:
imp_rf = res['NO_TOTAL']['imp_rf']
imp_gbst = res['NO_TOTAL']['imp_gbst']
print 'in rf not gbst', set(imp_rf.label) - set(imp_gbst.label)
print 'in gbst not rf', set(imp_gbst.label) - set(imp_rf.label)
data = [
    go.Bar(x=imp_rf.importance, y=imp_rf.label,  error_x=dict(array=imp_rf['std']), orientation='h', name='rf'),
    go.Bar(x=imp_gbst.importance, y=imp_gbst.label,  error_x=dict(array=imp_gbst['std']), orientation='h', name='gbst'),
    
]
layout = go.Layout(margin=go.Margin(l=300))
fig = go.Figure(data=data, layout=layout)

iplot(fig)

in rf not gbst set(['retail shop', 'd_node_ecc_SgAsNd', 'CRASHEVENTTYPES_ PENDING_INVESTIGATION', 'TRAFFIC SIGNAL MAIN'])
in gbst not rf set(['ud_clo_cntr_SgAsNd', 'ud_far_cntr_SgAsNd', 'd_page_rank_SgAsNd', 'ud_page_rank_SgAsNd'])


In [83]:
imp_rf = res['TOTAL']['imp_rf']
imp_gbst = res['TOTAL']['imp_gbst']
print 'in rf not gbst', set(imp_rf.label) - set(imp_gbst.label)
print 'in gbst not rf', set(imp_gbst.label) - set(imp_rf.label)

data = [
    go.Bar(x=imp_rf.importance, y=imp_rf.label,  error_x=dict(array=imp_rf['std']), orientation='h', name='rf'),
    go.Bar(x=imp_gbst.importance, y=imp_gbst.label,  error_x=dict(array=imp_gbst['std']), orientation='h', name='gbst'),
    
]
layout = go.Layout(margin=go.Margin(l=200))
fig = go.Figure(data=data, layout=layout)

iplot(fig)

in rf not gbst set(['d_node_ecc_SgAsNd'])
in gbst not rf set(['d_btw_cntr_SgAsNd'])
